<a href="https://colab.research.google.com/github/Prabhu-Tejas/GenAIclimbers/blob/main/GenAiclimbers_v1_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the Generative AI Hackthon Project.

Team name: GenAI Climbers


Project name: Insurance:Expidating document validation processing.

In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.5 MB/s eta 0:00:00


In [3]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00


In [4]:
#importing Libraries

import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

warnings.filterwarnings("ignore")
import PyPDF2

PROJECT_ID = "gen-genaiclimbers"  # @param {type:"string"}
REGION = "us-west1"

import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

vertex_llm_text = VertexAI(model_name="text-bison@001")
vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

In [ ]:
#data ingestion

pdf_loader = PyPDFLoader('/content/ShahRukh_MedicalReport.pdf')
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

In [ ]:
#Prompting

question = "Verify the hospital report, doctor report and the patient name and tell us whether the insurance claim can be approved?"
prompt_template = """Answer the question as precise as possible using the provided context. If the report is
                    not sufficient in the context, say "Claim is rejected" \n\n
                    Verify the doctor report \n
                    Verify the hospital report \n
                    Verify the names in doctor report , hospital report and verify the patient name
                    and tell in precise way\n
                    and display the patient name and company name \n
                    will the cost claimed in the insurance policy \n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
# we are using Stuffing type of LLM chain

stuff_chain = load_qa_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)

Stuffing Chain

The Stuffing chain serves as a solution for scenarios where the context length of the LLM is inadequate to handle extensive documents or a substantial amount of information. In such cases, a large document can be divided into smaller segments, and semantic search techniques can be employed to retrieve relevant documents based on the query. These retrieved documents are then “stuffed” into the LLM context, allowing for the generation of a response.

In [ ]:
from datetime import datetime
start = datetime.now()
stuff_answer = stuff_chain(
    {"input_documents": pages[1:4], "question": question}, return_only_outputs=True
)
end = datetime.now()
td = (end - start).total_seconds() * 10**3
print(f"The time of execution of above program is : {td:.03f}ms")

The time of execution of above program is : 1249.807ms


In [ ]:
pprint(stuff_answer)

{'output_text': "The insurance claim can be approved. The patient's name is "
                'Shah Rukh, and they are covered under the Capgemini group '
                'insurance policy. The surgery is necessary to repair and '
                "stabilize the fractured bone in the patient's right hand. The "
                'estimated cost of the surgery is INR 1,10,000/-, and the '
                'insurance policy covers this cost.'}
